# Time evolution of covid-19 cases in the United State (by county)

- Created by:
    - *Von P. Walden*
    - *Washington State University*
    - *Department of Civil and Environmental Engineering*
    - *28 March 2020*

In [1]:
import warnings
warnings.simplefilter('ignore')

In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
import requests
import hvplot.pandas

### Data Sources

- Special thanks to the New York Times for making county-level covid-19 data available at https://github.com/nytimes/covid-19-data
- Geographic data for US counties acquired from https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_20m.zip 
- FIPS table for US States acquired from https://gist.github.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53

In [18]:
US_County_Covid19_Data = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
dates = pd.to_datetime(US_County_Covid19_Data.date.unique())

In [19]:
gdf = gpd.read_file('cb_2018_us_county_20m/cb_2018_us_county_20m.shp')

In [20]:
fips = pd.read_csv('us-state-ansi-fips.csv', skiprows=1, dtype=str)
states = fips.stname.tolist()

In [23]:
options = [(i.strftime('%Y-%m-%d'), i) for i in dates]
daynum  = dates.strftime('%s').astype(int)

intSlide = widgets.IntSlider(
    value=daynum[-1],
    min=daynum[0],
    max=daynum[-1],
    step=1,
    description='Date:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False
)

@widgets.interact(state=states, date=intSlide)
def plotCases(state='Washington', date=daynum[-1]):
    dt       = pd.to_datetime(date, unit='s').strftime('%Y-%m-%d')
    fipsCode = fips[fips.stname==state].st.values[0];
    mx       = US_County_Covid19_Data.loc[US_County_Covid19_Data.state==state].cases.max()
    sm       = US_County_Covid19_Data.loc[US_County_Covid19_Data.state==state].loc[US_County_Covid19_Data.date==dt].cases.sum()
    covid19  = US_County_Covid19_Data.loc[US_County_Covid19_Data.state==state].loc[US_County_Covid19_Data.date==dt]
    data     = gdf[gdf['STATEFP']==fipsCode].set_index('NAME').join(covid19[covid19.state==state].set_index('county'))

    return data.hvplot(title = state + ' - ' + dt + '   Total reported cases: ' + str(sm), frame_width=600, geo=True, c='cases', cmap='Viridis', clim=(0, mx), xaxis=None, yaxis=None, hover_cols=['cases', 'NAME'])

interactive(children=(Dropdown(description='state', index=47, options=('Alabama', 'Alaska', 'Arizona', 'Arkans…